# mlflow를 활용한 훈련 기록 및 모델 서빙

### mlflow 설치

In [ ]:
# !pip install mlflow

### 터미널에 다음 명령어를 입려한뒤 mlflow gui 실행

In [ ]:
# 주피터 노트북(8888)과 포트가 겹치지 않도록 8889로 실행
# mlflow ui -h 0.0.0.0 -p 8889

### mlflow에 실험 초기화

In [ ]:
import mlflow

In [ ]:
try:
    #프로젝트 별로 이름을 다르게 가져가면서 실험들을 기록
    mlflow.create_experiment(name='pytorch-test')
except:
    print('Exist experiment')

mlflow.set_experiment('pytorch-test')

In [ ]:
#mlflow에 기록할 준비
mlflow.start_run()

In [ ]:
#현재 모델/훈련의 버전 지정
mlflow.set_tag('version', '0.1')

In [ ]:
# 하이퍼 파라미터 설정
params = {
    'learning_rate' : 0.01,
    'epochs' : 100,
    'batch_size' : 128
}

In [ ]:
# mlflow에 현재 실험의 하이퍼파라미터 등록
mlflow.log_params(params)

### 가상의 데이터 및 모델 준비

In [ ]:
import torch
from torch import nn
from torch.nn import Sequential
from torch.optim import Adam

In [ ]:
model = Sequential(
    nn.Linear(2,512),
    nn.ReLU(),
    nn.Linear(512, 1),
    nn.Sigmoid())


In [ ]:
loss = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=params['learning_rate'])

## 모델 훈련(train)

In [ ]:
# 임시 데이터 생성
import numpy as np
X = torch.tensor(np.random.rand(300, 2), dtype=torch.float32)
Y = torch.tensor(np.random.rand(300)[:, np.newaxis], dtype=torch.float32)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
train_dataset, valid_dataset = random_split(TensorDataset(X, Y), [270, 30])
train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=params['batch_size'])

훈련 과정중에 mlflow.log_metric 함수를 이용해 성능 기록

In [ ]:
for epoch in range(params['epochs']):
    epoch_loss = 0.
    for train_x, train_y in train_loader:
        optimizer.zero_grad()
        y_pred = model(train_x)
        
        _loss = loss(y_pred, train_y)
        
        mlflow.log_metric('batch_loss', _loss.item(), step=epoch)
        _loss.backward()
        optimizer.step()
        
        epoch_loss += _loss.item()
    
    ##MLFLOW에 기록
    mlflow.log_metric('loss', epoch_loss / len(train_dataset), step=epoch)
    
    with torch.no_grad():
        valid_loss = 0.
        for valid_x, valid_y in valid_loader:
            y_pred = model(valid_x)
            _valid_loss = loss(y_pred, valid_y)
            
            mlflow.log_metric('batch_val_loss', _valid_loss.item(), step=epoch)
            valid_loss += _valid_loss.item()
        
        ##MLFLOW에 기록 
        mlflow.log_metric('val_loss', valid_loss / len(valid_dataset), step=epoch)
    
#     torch.save(model.state_dict(), 'weights.pt')

### 훈련한 모델을 mlflow에 저장

In [ ]:
mlflow.pytorch.log_model(model, "save_model")

In [ ]:
#mlflow 기록 종료
mlflow.end_run()

## 모델 서빙
mlflow gui에 접속하여 방금 훈련을 마친 모델의 실험 기록을 확인 한우에, 모델 디렉토리를 복사
- 모델 디렉토리 예: file:///Users/Tom/projects/9rkd/mlruns/1/a69f1d42be0e404097c19e3d2cd7fb7a/artifacts/save_model

터미널에 아래 명령어 입력

In [ ]:
# mlflow GUI와 포트가 겹치지 않도록 8890으로 실행

# mlflow models serve -m <saved-model-dir> --no-conda -h 0.0.0.0 -p 8890

### curl로 요청을 보낼시
기본적으로 http://[HOST]:[PORT]/invocations와 같이 'invocations'으로 resource path가 지정됨

터미널에서 다음 명령어를 입력한뒤 반환되는 값 확인

In [ ]:
# curl http://127.0.0.1:8890/invocations -H 'Content-Type: application/json' -d '{"columns": ["dense_input", "dense_input"],"data": [[0.1, 0.2]]}'

### python에서 보내는 경우

In [ ]:
import requests, json
url = 'http://localhost:8890/invocations'
data = {
    "columns": ["dense_input", "dense_input"],
    "data": [[0.1, 0.2]]
}
headers = {
    'content-type':'application/json'
}
res = requests.post(url, headers=headers, data=json.dumps(data))
res.text